In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, IBMQ, BasicAer
import math

In [2]:
# Set up the program
alice = QuantumRegister(1, name='alice')
fiber = QuantumRegister(1, name='fiber')
bob   = QuantumRegister(1, name='bob')

In [3]:
alice_had = ClassicalRegister(1, name='ahad')
alice_val = ClassicalRegister(1, name='aval')
fiber_val = ClassicalRegister(1, name='fval')
bob_had   = ClassicalRegister(1, name='bhad')
bob_val   = ClassicalRegister(1, name='bval')


In [4]:
qc = QuantumCircuit(alice, fiber, bob, alice_had, alice_val, fiber_val, bob_had, bob_val)

In [5]:
# Use Alice's QPU to generate two random bits
qc.reset(alice)          # write the value 0
qc.h(alice)
qc.measure(alice, alice_had)
qc.reset(alice)          # write the value 0
qc.h(alice)
qc.measure(alice, alice_val)

# Prepare Alice's qubit
qc.reset(alice)          # write the value 0
qc.x(alice).c_if(alice_val, 1)
qc.h(alice).c_if(alice_had, 1)

# Send the qubit!
qc.swap(alice, fiber)

# Activate the spy
spy_is_present = True
if spy_is_present:
    qc.barrier()
    spy_had = True
    if spy_had:
        qc.h(fiber)
    qc.measure(fiber, fiber_val)
    qc.reset(fiber)
    qc.x(fiber).c_if(fiber_val, 1)
    if spy_had:
        qc.h(fiber)

qc.barrier()

# Use Bob's QPU to generate a random bit
qc.reset(bob)
qc.h(bob)
qc.measure(bob, bob_had)

# Receive the qubit!
qc.swap(fiber, bob)
qc.h(bob).c_if(bob_had, 1)
qc.measure(bob, bob_val)

In [6]:
qc.draw()

┌───┐┌─┐     ┌───┐┌─┐      ┌───┐  ┌───┐     ░              »
alice_0: ─|0>─┤ H ├┤M├─|0>─┤ H ├┤M├─|0>──┤ X ├──┤ H ├──X──░──────────────»
              └───┘└╥┘     └───┘└╥┘      └─╥─┘  └─╥─┘  │  ░ ┌───┐┌─┐     »
fiber_0: ───────────╫────────────╫─────────╫──────╫────X──░─┤ H ├┤M├─|0>─»
                    ║            ║         ║      ║       ░ └───┘└╥┘     »
  bob_0: ───────────╫────────────╫─────────╫──────╫───────░───────╫──────»
                    ║            ║         ║   ┌──╨──┐    ░       ║      »
 ahad: 1/═══════════╩════════════╬═════════╬═══╡ = 1 ╞════════════╬══════»
                    0            ║      ┌──╨──┐└─────┘            ║      »
 aval: 1/════════════════════════╩══════╡ = 1 ╞═══════════════════╬══════»
                                 0      └─────┘                   ║      »
 fval: 1/═════════════════════════════════════════════════════════╩══════»
                                                                  0      »
 bhad: 1/════════════════════════════════════════════════════════════════»
                                                                         »
 bval: 1/════════════════════════════════════════════════════════════════»
                                                                         »
«                      ░                           
«alice_0: ─────────────░───────────────────────────
«          ┌───┐ ┌───┐ ░                           
«fiber_0: ─┤ X ├─┤ H ├─░───────────────X───────────
«          └─╥─┘ └───┘ ░      ┌───┐┌─┐ │  ┌───┐ ┌─┐
«  bob_0: ───╫─────────░──|0>─┤ H ├┤M├─X──┤ H ├─┤M├
«            ║         ░      └───┘└╥┘    └─╥─┘ └╥┘
« ahad: 1/═══╬══════════════════════╬═══════╬════╬═
«            ║                      ║       ║    ║ 
« aval: 1/═══╬══════════════════════╬═══════╬════╬═
«         ┌──╨──┐                   ║       ║    ║ 
« fval: 1/╡ = 1 ╞═══════════════════╬═══════╬════╬═
«         └─────┘                   ║    ┌──╨──┐ ║ 
« bhad: 1/══════════════════════════╩════╡ = 1 ╞═╬═
«                                   0    └─────┘ ║ 
« bval: 1/═══════════════════════════════════════╩═
«                                                0

In [7]:
'''
OPENQASM 2.0;
include "qelib1.inc";

qreg alice[1];
qreg connection[1];
qreg bob[1];
creg a1[1];
creg a2[1];
creg con[1];
creg b1[2];
creg b2[2];

reset alice[0];
h alice[0];
measure alice[0] -> a1[0];
reset alice[0];
h alice[0];
measure alice[0] -> a2[0];
reset alice[0];

if(a1==1) x alice[0];
if(a2==1) h alice[0];

swap  alice[0], connection[0];
h connection[0];
measure connection[0] -> con[0];
reset connection[0];
if (con==1) x connection[0];
h connection[0];

reset bob[0];
h bob[0];
measure bob[0] -> b1[0];

swap connection[0], bob[0];
if(b1==1) h bob[0];
measure bob[0] -> b2[0];

'''

'\nOPENQASM 2.0;\ninclude "qelib1.inc";\n\nqreg alice[1];\nqreg connection[1];\nqreg bob[1];\ncreg a1[1];\ncreg a2[1];\ncreg con[1];\ncreg b1[2];\ncreg b2[2];\n\nreset alice[0];\nh alice[0];\nmeasure alice[0] -> a1[0];\nreset alice[0];\nh alice[0];\nmeasure alice[0] -> a2[0];\nreset alice[0];\n\nif(a1==1) x alice[0];\nif(a2==1) h alice[0];\n\nswap  alice[0], connection[0];\nh connection[0];\nmeasure connection[0] -> con[0];\nreset connection[0];\nif (con==1) x connection[0];\nh connection[0];\n\nreset bob[0];\nh bob[0];\nmeasure bob[0] -> b1[0];\n\nswap connection[0], bob[0];\nif(b1==1) h bob[0];\nmeasure bob[0] -> b2[0];\n\n'